In [1]:
import pandas as pd
import numpy as np


## Unbalanced Data

In [2]:
#Select feature to predict on

feature = 'Gender'
data_1_2_3 = 3


In [3]:
#Choosing file path


if data_1_2_3 ==1:
    file_location = 'D:\\Victor Yotov\\ITU\\3rd Semester\\Research Project\\data\\Directors_Dataset\\1_Director_Data_Mar-21-2020.csv'
elif data_1_2_3 == 2:
    file_location = 'D:\\Victor Yotov\\ITU\\3rd Semester\\Research Project\\data\\Directors_Dataset\\2_Director_Data_Enhanced_May-10-2020.csv'
elif data_1_2_3 == 3:
    file_location = 'D:\\Victor Yotov\\ITU\\3rd Semester\\Research Project\\data\\Directors_Dataset\\3_USA_Director_Specific_dataset_May-15-2020.csv'
else:
    raise ValueError('Wrong INTEGER provided.') 
    
data = pd.read_csv(file_location)

data.shape

(1215, 56)

In [4]:
file_location

'D:\\Victor Yotov\\ITU\\3rd Semester\\Research Project\\data\\Directors_Dataset\\3_USA_Director_Specific_dataset_May-15-2020.csv'

In [5]:
data.head(5)

,director_ids,director_name,average_rating_value,average_rating_count,movies_produced,career_Length,CareerStartYear,Gender,birthYear,median_inter_event_time,...,total_genre_Romance,total_genre_Sci_Fi,total_genre_Short,total_genre_Sport,total_genre_Talk_Show,total_genre_Thriller,total_genre_War,total_genre_Western,average_movie_per_year,num_genres_directed
0,nm0000040,Stanley Kubrick,7.30,231646.00,1,49,1951,Male,1928,0.0,...,0,0,0,0,0,1,0,0,0.02,3
1,nm0000095,Woody Allen,6.91,64233.13,15,51,1966,Male,1935,1.0,...,8,0,0,0,0,1,0,0,0.29,7
2,nm0000104,Antonio Banderas,6.30,5964.00,1,1,1999,Male,1960,0.0,...,0,0,0,0,0,0,0,0,1.00,3
3,nm0000106,Drew Barrymore,6.90,56419.00,1,6,2004,Male,1975,0.0,...,0,0,0,1,0,0,0,0,0.17,2
4,nm0000108,Luc Besson,6.30,89739.00,1,33,1981,Male,1959,0.0,...,0,0,0,0,0,1,0,0,0.03,3


In [6]:
#If 1 ---- MovieDataset 1
if data_1_2_3 == 1:
     data = data.drop(columns=['director_ids', 'director_name','primaryProfession', 'knownForTitles'])
    
elif data_1_2_3 == 2 or data_1_2_3 ==3:
    data = data.drop(columns=['director_ids', 'director_name'])

    
# data.drop(columns=['director_ids', 'director_names','movie_id','title','primaryProfession', 'knownForTitles','date_published', 'release_date'],inplace=True)
# data = data.drop(columns=['director_ids', 'director_names', 'movie_id', 'title',
#        'release_date', 'Film', 'Name'])

In [7]:
data['Gender'] = data['Gender'].map({'Male':0,'Female':1})



In [8]:
data['Gender'].value_counts()

0    1038
1     177
Name: Gender, dtype: int64

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from string import ascii_uppercase
import seaborn as sn

def report_accuracy_RF(dataframe, predicted_feature, numberEstimators=100):
    #Splitting X and y
    X = dataframe[dataframe.columns.difference([predicted_feature],sort=False)]
    y = dataframe[predicted_feature]
    
    #Initialize Random Forest
    rnd_forest = RandomForestClassifier(n_estimators=numberEstimators, class_weight='balanced', random_state=52)

#     rnd_forest = RandomForestClassifier(n_estimators=numberEstimators, class_weight='balanced', oob_score=True)
#     rnd_forest = RandomForestClassifier(n_estimators=numberEstimators, oob_score=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    rnd_forest.fit(X_train,y_train.values.ravel())
    score = rnd_forest.score(X_test,y_test)
    
    return score, rnd_forest,X, y ,X_train, X_test, y_train, y_test
    

In [10]:
def create_cnf_matrix(y_test, X_test, model):
    
    y_true = y_test.copy()
    y_predict = model.predict(X_test)
    
### C_0_0 = TP - male being classified as a male
### C_0_1 = Males being identified as femals
### C_1_0 = Female being identified as male
### C_1_1 = TN female identified as female  

    #define the order of classes appearing in the matrix
    labels = ['Terrible','Poor','Average','Excellent']
    cnf_matrix = confusion_matrix(y_true, y_predict,  labels=labels)
    
    return cnf_matrix
    

In [11]:
def calculate_confusion_matrix_metrics(confusion_matrix):
    #source:
    #https://stackoverflow.com/questions/50666091/true-positive-rate-and-false-positive-rate-tpr-fpr-for-multi-class-data-in-py
    
    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    
    return TN, FP, FN, TP

In [12]:
def print_accuracy_feature_importance(score, X, model):
    
    print(f'The accuracy of the model is equal to {score} %')
    print()
    print(X.columns)
    print()
    print(model.feature_importances_)
    

In [13]:
def customized_classification_report(y_true,y_pred, AUCROC ):

    cols_to_use_classification_report = ['0','1','accuracy']

    reportGBClassification = classification_report(y_true,y_pred, output_dict=True)

    reportDF = pd.DataFrame(reportGBClassification, columns = cols_to_use_classification_report)[0:3]
    reportDF.columns = ['Male','Female','accuracy']

    reportDF[['Male','Female']] = reportDF[['Male','Female']].round(2)
    accuracyRow = reportDF['accuracy'].unique().tolist()

    reportDF.drop(columns='accuracy',inplace= True)

    reportDF.loc[len(reportDF)] = 2 * accuracyRow
    reportDF.loc[len(reportDF)] = 2 * [AUCROC]

    reportDF = reportDF.rename(index = {3:'accuracy',4:'AUC ROC'})
    return reportDF


In [14]:
score, rnd_forest,X, y ,X_train, X_test, y_train, y_test = report_accuracy_RF(data,"Gender")
print_accuracy_feature_importance(score,X, rnd_forest)


The accuracy of the model is equal to 0.8559670781893004 %

Index(['average_rating_value', 'average_rating_count', 'movies_produced',
       'career_Length', 'CareerStartYear', 'birthYear',
       'median_inter_event_time', 'average_inter_event_time',
       'total_review_count_user', 'total_review_count_critic',
       'total_metascore', 'median_metascore', 'total_awards_won',
       'total_awards_nominated', 'total_gross_value', 'total_budget_value',
       'average_runtime_of_movie', 'director_profession_primary',
       'director_profession_secondary', 'director_profession_tertiary',
       'director_known_titles_count',
       'director_known_titles_average_rating_value', 'total_genre_Action',
       'total_genre_Adult', 'total_genre_Adventure', 'total_genre_Animation',
       'total_genre_Biography', 'total_genre_Comedy', 'total_genre_Crime',
       'total_genre_Documentary', 'total_genre_Drama', 'total_genre_Family',
       'total_genre_Fantasy', 'total_genre_Film_Noir', 'total_

In [15]:
y_predict= rnd_forest.predict(X_test)


In [16]:
from sklearn.metrics import classification_report


print(classification_report(y_test,y_predict,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92       208
           1       0.00      0.00      0.00        35

    accuracy                           0.86       243
   macro avg       0.43      0.50      0.46       243
weighted avg       0.73      0.86      0.79       243



C:\Users\VictorY\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from sklearn.metrics import roc_auc_score


y_prob = rnd_forest.predict_proba(X_test)
ovo_macro_roc_auc = roc_auc_score(y_test, y_predict)
print( "\nArea Under the Curve - ROC (OVO , Macro) : ",ovo_macro_roc_auc)


Area Under the Curve - ROC (OVO , Macro) :  0.5


In [18]:
y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int64)

In [19]:
customized_classification_report(y_test,y_predict, ovo_macro_roc_auc )

C:\Users\VictorY\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Male,Female
precision,0.860000,0.000000
recall,1.000000,0.000000
f1-score,0.920000,0.000000
accuracy,0.855967,0.855967
AUC ROC,0.500000,0.500000


In [20]:
data.shape

(1215, 54)

### Balanced on Gender

In [21]:
data3 = pd.read_csv(file_location)
data3['Gender'] = data3['Gender'].map({'Male':0, 'Female':1})

In [22]:
# Generic function to equally max split the dataframe with respect to the column Name, in our case the Gender column.

def max_equal_split_on_columnName(df,columnName):
    df1 = pd.DataFrame()
    for i in df[columnName].value_counts().index:
        df1 = df1.append(df[(df[columnName]==i)]
                         .sample(n=df[columnName]
                                 .value_counts()
                                 .values.min()
                                 ,replace = False)
                        )        
    return df1.reset_index(drop=True)

In [23]:
data3.Gender.value_counts()

0    1038
1     177
Name: Gender, dtype: int64

In [24]:
balanced_data = max_equal_split_on_columnName(data3, 'Gender')

if data_1_2_3 == 1:
    
    balanced_data=  balanced_data.drop(columns=['director_ids', 'director_name','primaryProfession', 'knownForTitles'])
    
elif data_1_2_3 == 2 or data_1_2_3 ==3:
    balanced_data = balanced_data.drop(columns=['director_ids', 'director_name'])



In [25]:
balanced_data.shape

(354, 54)

In [26]:
balanced_data.Gender.value_counts()

1    177
0    177
Name: Gender, dtype: int64

In [27]:
balanced_data

,average_rating_value,average_rating_count,movies_produced,career_Length,CareerStartYear,Gender,birthYear,median_inter_event_time,average_inter_event_time,total_review_count_user,...,total_genre_Romance,total_genre_Sci_Fi,total_genre_Short,total_genre_Sport,total_genre_Talk_Show,total_genre_Thriller,total_genre_War,total_genre_Western,average_movie_per_year,num_genres_directed
0,5.80,80497.00,1,14,1996,0,1969,0.0,0.00,355,...,0,1,0,0,0,0,0,0,0.07,3
1,3.50,10239.00,1,8,2002,0,1980,0.0,0.00,50,...,0,0,0,0,0,0,0,0,0.12,3
2,6.90,21875.00,1,14,2003,0,1980,0.0,0.00,96,...,0,0,0,0,0,0,0,0,0.07,3
3,5.38,23847.50,4,23,1988,0,1948,2.0,2.67,405,...,0,0,0,0,0,0,0,0,0.17,6
4,6.70,11782.00,1,6,2002,0,1969,0.0,0.00,52,...,0,0,0,0,0,0,0,0,0.17,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,7.00,109009.00,1,22,1990,1,1959,0.0,0.00,182,...,1,0,0,0,0,0,0,0,0.05,2
350,6.77,69357.33,3,9,2005,1,1959,2.0,2.00,736,...,0,0,0,0,0,1,0,0,0.33,3
351,6.35,32252.00,2,19,1998,1,1972,2.0,2.00,274,...,0,0,0,0,0,0,0,0,0.11,6
352,6.35,19370.00,2,5,1999,1,1960,4.0,4.00,378,...,2,0,0,0,0,0,0,0,0.40,3


In [28]:
score3, rnd_forest3,X3, y3 ,X_train3, X_test3, y_train3, y_test3 = report_accuracy_RF(balanced_data,"Gender")

print(f"The accuracy of the model is {score3}.")
print()
print(f'The input features included are {X3.columns} ')
print()
print(f'The corresponding feature importance is{rnd_forest3.feature_importances_}')

The accuracy of the model is 0.5915492957746479.

The input features included are Index(['average_rating_value', 'average_rating_count', 'movies_produced',
       'career_Length', 'CareerStartYear', 'birthYear',
       'median_inter_event_time', 'average_inter_event_time',
       'total_review_count_user', 'total_review_count_critic',
       'total_metascore', 'median_metascore', 'total_awards_won',
       'total_awards_nominated', 'total_gross_value', 'total_budget_value',
       'average_runtime_of_movie', 'director_profession_primary',
       'director_profession_secondary', 'director_profession_tertiary',
       'director_known_titles_count',
       'director_known_titles_average_rating_value', 'total_genre_Action',
       'total_genre_Adult', 'total_genre_Adventure', 'total_genre_Animation',
       'total_genre_Biography', 'total_genre_Comedy', 'total_genre_Crime',
       'total_genre_Documentary', 'total_genre_Drama', 'total_genre_Family',
       'total_genre_Fantasy', 'total_gen

In [29]:
# y_true3 = y_test3.copy()
y_predict3 = rnd_forest3.predict(X_test3)


from sklearn.metrics import classification_report


print(classification_report(y_test3,y_predict3,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59        36
           1       0.58      0.60      0.59        35

    accuracy                           0.59        71
   macro avg       0.59      0.59      0.59        71
weighted avg       0.59      0.59      0.59        71



In [30]:
# y_prob3 = rnd_forest3.predict_proba(X_test3)
ovo_macro_roc_auc3 = roc_auc_score(y_test3, y_predict3)
print( "\nArea Under the Curve - ROC (OVO , Macro) : ",ovo_macro_roc_auc3)


Area Under the Curve - ROC (OVO , Macro) :  0.5916666666666666


In [31]:
rnd_forest3

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=52, verbose=0,
                       warm_start=False)

In [32]:
customized_classification_report(y_test3,y_predict3, ovo_macro_roc_auc3)

,Male,Female
precision,0.600000,0.580000
recall,0.580000,0.600000
f1-score,0.590000,0.590000
accuracy,0.591549,0.591549
AUC ROC,0.591667,0.591667


## Summary Reports

### Unbalanced

In [33]:
customized_classification_report(y_test,y_predict, ovo_macro_roc_auc )

C:\Users\VictorY\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Male,Female
precision,0.860000,0.000000
recall,1.000000,0.000000
f1-score,0.920000,0.000000
accuracy,0.855967,0.855967
AUC ROC,0.500000,0.500000


###  Balanced

In [34]:
customized_classification_report(y_test3,y_predict3, ovo_macro_roc_auc3)

,Male,Female
precision,0.600000,0.580000
recall,0.580000,0.600000
f1-score,0.590000,0.590000
accuracy,0.591549,0.591549
AUC ROC,0.591667,0.591667
